In [53]:
#from rdfpandas.graph import to_dataframe
import shared.config as config
import pymongo
import numpy as np
import pandas as pd
import json
import re
import shared.const as const
import difflib

#import rdflib
from ipywidgets import interact, Dropdown
from IPython.display import display
import ipywidgets as widgets

import sqlalchemy as db
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, inspect
from sqlalchemy.engine import reflection

pd.set_option('display.max_rows', 100)
pd.set_option('max_colwidth', 400)

inputExcel = '/input/referentietabellen/abr_versie_01122018.xlsx'
outputExcel = '/input/referentietabellen/abr_versie_01122018_input.xlsx'
#ABRTurtle = './work/datasets/abr.ttl'

In [21]:
myclient = pymongo.MongoClient(str(config.MONGO_URI))
stagingDb = myclient[str(config.DB_STAGING)]
analyseDb = myclient[str(config.DB_ANALYSE)]
stagingCol = stagingDb[config.COLL_PLAATJES]
stagingOud = stagingDb[config.COLL_STAGING_OUD]
stagingNieuw = stagingDb[config.COLL_STAGING_NIEUW]
analyseCol = analyseDb[config.COLL_ANALYSE]
analyseColClean = analyseDb[config.COLL_ANALYSE_CLEAN]

metaCollection = stagingDb['Kolominformatie']

In [3]:
#g = rdflib.Graph()
#g.parse(ABRTurtle, format = 'ttl')
#df_import = to_dataframe(g)
#df.to_csv('test.csv', index = True, index_label = "@id")
#df_import

In [4]:
df_abr = pd.read_excel(inputExcel)
df_abr.reset_index(inplace=True)
df_abr.rename(columns={"index": "ID"}, inplace=True)
df_abr.dropna(subset=['uri'])
df_abr

,ID,uri,scheme,concept,concept.1,concept.2,concept.3,concept.4,concept.5,concept.6,...,https://data.cultureelerfgoed.nl/vocab/id/abr#hasBeginYear,https://data.cultureelerfgoed.nl/vocab/id/abr#hasEndYear,https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode,https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode,primat,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl
0,0,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,Algemene Lijsten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,NaN,bronhouder,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380,NaN,NaN,Stichting Infrastructuur Kwaliteitsborging Bodembeheer,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09,NaN,NaN,Rijksdienst voor het Cultureel Erfgoed,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b,NaN,concept status,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4853,4853,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a,NaN,NaN,stuifzandvlakte,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VSZ,NaN,NaN,NaN,NaN
4854,4854,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f,NaN,NaN,heuvelrug door tectoniek ontstaan,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,RHE,NaN,NaN,NaN,NaN
4855,4855,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be,NaN,NaN,veenvlakte (ontgonnen),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,VOV,NaN,NaN,NaN,NaN
4856,4856,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23,NaN,NaN,oeverwal,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,ROE,NaN,NaN,NaN,NaN


In [5]:
def getCode(row):
    lst_codes = ['https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode', 
                 'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl', 
                 'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode', 
                 'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode@nl']
    for n in range(0, len(lst_codes)-1):
        if pd.notnull(row[lst_codes[n]]):
            return (row[lst_codes[n]])
    return None


lst = []
lst_cnc = []
lst_cd = []
lst_id = []

for index, row in df_abr.iterrows():
    if pd.notnull(row.scheme):
        lst.clear()
        lst.append(row.uri) 
        lst_id.clear()
        lst_id.append(row.ID) 
        lst_cnc.clear()
        lst_cnc.append(row.scheme)
        lst_cd.clear()
        lst_cd.append(getCode(row)) 
    elif pd.notnull(row.concept):
        lst = lst[:1]
        lst.append(row.uri) 
        lst_id = lst_id[:1]
        lst_id.append(row.ID) 
        lst_cnc = lst_cnc[:1]
        lst_cnc.append(row.concept)
        lst_cd = lst_cd[:1]
        lst_cd.append(getCode(row)) 
    else:
        rng = range(1,7)
        for n in rng:
            if pd.notnull(row['concept.' + str(n)]):
                lst = lst[:n+1]
                lst.append(row.uri) 
                lst_id = lst_id[:n+1]
                lst_id.append(row.ID) 
                lst_cnc = lst_cnc[:n+1]
                lst_cnc.append(row['concept.' + str(n)])
                lst_cd = lst_cd[:n+1]
                lst_cd.append(getCode(row))
    
    #print(lst)
    df_abr.at[index, 'uris'] = lst
    df_abr.at[index, 'parent_uri'] = lst[len(lst)-2] if len(lst)-2 > -1 else None
    df_abr.at[index, 'concept'] = lst_cnc[len(lst_cnc)-1] if len(lst_cnc)-1 > -1 else None
    df_abr.at[index, 'parent_Concept'] = lst_cnc[len(lst_cnc)-2] if len(lst_cnc)-2 > -1 else None
    df_abr.at[index, 'code'] = lst_cd[len(lst_cd)-1] if len(lst_cd)-1 > -1 else None
    df_abr.at[index, 'parent_code'] = lst_cd[len(lst_cd)-2] if len(lst_cd)-2 > -1 else None
    df_abr.at[index, 'parentID'] = lst_id[len(lst_id)-2] if len(lst_id)-2 > -1 else None
        
    
parent = 'https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10'
df_abr[df_abr.uris.apply(lambda x: parent in x)].head()

,ID,uri,scheme,concept,concept.1,concept.2,concept.3,concept.4,concept.5,concept.6,...,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification,https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl,uris,parent_uri,parent_Concept,code,parent_code,parentID
31,31,https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,NaN,materialen,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10]",https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed,Archeologisch Basis Register,NaN,NaN,30.0
32,32,https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen,NaN,vervallen materialen,vervallen materialen,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen]",https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10,materialen,NaN,NaN,31.0
33,33,https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,NaN,magering,NaN,magering,NaN,NaN,NaN,NaN,...,NaN,NaN,was ABR specifiek niet in KNA.,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e]",https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen,vervallen materialen,KMA,NaN,32.0
34,34,https://data.cultureelerfgoed.nl/term/id/abr/134b1acb-18ec-455a-bc0b-f237cfb3b2f9,NaN,calciet,NaN,NaN,calciet,NaN,NaN,NaN,...,NaN,NaN,vervallen,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e, https://data.cultureelerfgoed.nl/term/id/abr/134b1acb-18ec-455a-bc0b-f237cfb3b2f9]",https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,magering,KMC,KMA,33.0
35,35,https://data.cultureelerfgoed.nl/term/id/abr/bfd2a9c3-9f59-472a-ad5b-7ba32846c565,NaN,organische magering,NaN,NaN,organische magering,NaN,NaN,NaN,...,NaN,NaN,vervallen,NaN,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10, https://data.cultureelerfgoed.nl/term/id/abr/vervallen_materialen, https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e, https://data.cultureelerfgoed.nl/term/id/abr/bfd2a9c3-9f59-472a-ad5b-7ba32846c565]",https://data.cultureelerfgoed.nl/term/id/abr/3c6900e6-a665-4603-b123-4dfb84afdc6e,magering,KMO,KMA,33.0


In [6]:
df_abr.rename(columns={"ID": "primary_key", "altLabel@nl": "label", "scopeNote@nl": 'note', "https://data.cultureelerfgoed.nl/vocab/id/abr#hasBeginYear":"beginjaar", "https://data.cultureelerfgoed.nl/vocab/id/abr#hasEndYear": "eindjaar", 
                       "https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId": "bron", "https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification": "opmerkingen"}, inplace=True)
df_abr.columns

Index(['primary_key', 'uri', 'scheme', 'concept', 'concept.1', 'concept.2',
       'concept.3', 'concept.4', 'concept.5', 'concept.6', 'concept.7',
       'label', 'hiddenLabel@nl', 'type', 'note', 'beginjaar', 'eindjaar',
       'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode',
       'https://data.cultureelerfgoed.nl/vocab/id/abr#hasSpecificAbrCode@nl',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasCode', 'primat',
       'bron', 'https://data.cultureelerfgoed.nl/vocab/id/rce#hasSourceId@nl',
       'opmerkingen',
       'https://data.cultureelerfgoed.nl/vocab/id/rce#hasSpecification@nl',
       'uris', 'parent_uri', 'parent_Concept', 'code', 'parent_code',
       'parentID'],
      dtype='object')

In [7]:
df_abr = df_abr[['primary_key', 'concept', 'code', 'label', 'note', 'beginjaar', 'eindjaar', 'bron', 'opmerkingen', 'parentID', 'uri', 'uris']]
df_abr

,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,parentID,uri,uris
0,0,Algemene Lijsten,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten,https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten
1,1,bronhouder,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4]"
2,2,Stichting Infrastructuur Kwaliteitsborging Bodembeheer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4, https://data.cultureelerfgoed.nl/term/id/abr/17e56d62-b1f2-42e1-afbf-578d7b018380]"
3,3,Rijksdienst voor het Cultureel Erfgoed,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/bc241422-ee0d-4c44-aead-94a25b5e9fa4, https://data.cultureelerfgoed.nl/term/id/abr/1576adc6-1af1-44d0-9d46-593eb6bcad09]"
4,4,concept status,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b,"[https://data.cultureelerfgoed.nl/term/id/abr/AlgemeneLijsten, https://data.cultureelerfgoed.nl/term/id/abr/d159858d-d746-4de5-98be-82cc5b697f6b]"
...,...,...,...,...,...,...,...,...,...,...,...,...
4853,4853,stuifzandvlakte,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/0ca36c41-de5b-4ba1-90b3-c0130c81787a]"
4854,4854,heuvelrug door tectoniek ontstaan,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/fe9a85f8-d7b2-4314-86b6-e8ed7866624f]"
4855,4855,veenvlakte (ontgonnen),None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/6acfbc78-488f-4b05-b86e-a2c9903b95be]"
4856,4856,oeverwal,None,NaN,NaN,NaN,NaN,NaN,NaN,4801.0,https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/82638dc4-8095-4c54-a7ff-49ac32ad5361, https://data.cultureelerfgoed.nl/term/id/abr/c4dff91e-2427-48d7-829a-2816711bdd23]"


In [8]:
df_abr.to_excel(outputExcel)

In [9]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT * from "Def_ABR"', connection)

df

,herkomst,key,soort,brondata,_id,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,uri,parentID,uris
0,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/0012691a-dabc-45ce-bdfd-0cd4b20b464a,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e62fd'), 'Unnamed: 0': 2506, 'primary_key': 2506, 'concept': 'Fles - Trechterhals:Isings 14', 'code': 'TRECHTHL', 'label': 'Fles - Trechterhals:Isings 133', 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 2489.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/0012691a-dabc-45ce-bdfd-0cd4b20b464a', 'uris': ...",63754cb8b09de9b0cb2e62fd,2506,Fles - Trechterhals:Isings 14,TRECHTHL,Fles - Trechterhals:Isings 133,None,NaN,NaN,,None,https://data.cultureelerfgoed.nl/term/id/abr/0012691a-dabc-45ce-bdfd-0cd4b20b464a,2489.0,None
1,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/00132370-76ed-4e1e-baa8-830296362764,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5d63'), 'Unnamed: 0': 1072, 'primary_key': 1072, 'concept': 'Cilindrische Rand', 'code': nan, 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 1026.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/00132370-76ed-4e1e-baa8-830296362764', 'uris': ""['https://data.cultureelerfgoed.nl/term/id/abr/...",63754cb8b09de9b0cb2e5d63,1072,Cilindrische Rand,,,None,NaN,NaN,,None,https://data.cultureelerfgoed.nl/term/id/abr/00132370-76ed-4e1e-baa8-830296362764,1026.0,None
2,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/001ca068-eee2-456f-acc3-e16ffc001597,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5cbb'), 'Unnamed: 0': 897, 'primary_key': 897, 'concept': 'Schaatsmotief', 'code': nan, 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 836.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/001ca068-eee2-456f-acc3-e16ffc001597', 'uris': ""['https://data.cultureelerfgoed.nl/term/id/abr/b6df784...",63754cb8b09de9b0cb2e5cbb,897,Schaatsmotief,,,None,NaN,NaN,,None,https://data.cultureelerfgoed.nl/term/id/abr/001ca068-eee2-456f-acc3-e16ffc001597,836.0,None
3,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/00499557-440f-4fb4-b618-e0d2cbd24039,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e60a1'), 'Unnamed: 0': 1902, 'primary_key': 1902, 'concept': 'Angelsaksisch (Afgerond) Biconisch Aw. - Plettke C1-3:Knikwandschaal', 'code': 'PLEC1-3', 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 1899.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/00499557-440f-4fb4-b618-e0d2cbd24039',...",63754cb8b09de9b0cb2e60a1,1902,Angelsaksisch (Afgerond) Biconisch Aw. - Plettke C1-3:Knikwandschaal,PLEC1-3,,None,NaN,NaN,,None,https://data.cultureelerfgoed.nl/term/id/abr/00499557-440f-4fb4-b618-e0d2cbd24039,1899.0,None
4,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/00503008-1184-4df4-96b7-c4eafb7dc143,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e6140'), 'Unnamed: 0': 2061, 'primary_key': 2061, 'concept': 'Inktpot - Type A', 'code': nan, 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 2059.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/00503008-1184-4df4-96b7-c4eafb7dc143', 'uris': ""['https://data.cultureelerfgoed.nl/term/id/abr/b...",63754cb8b09de9b0cb2e6140,2061,Inktpot - Type A,,,None,NaN,NaN,,None,https://data.cultureelerfgoed.nl/term/id/abr/00503008-1184-4df4-96b7-c4eafb7dc143,2059.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4833,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/xxx974b981a-e1fe-4b8c-86ea-b859fd31af6a,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5dee'), 'Unnamed: 0': 1211, 'primary_key': 1211, 'concept': 'Ed50_Gms', 'code': nan, 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 1209.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/xxx974b981a-e

In [24]:
engine = create_engine(config.SQLALCHEMY_DATABASE_URI)
with engine.connect() as connection:
    df = pd.read_sql_query('SELECT * from "Def_ABR" WHERE "parentID" = 31', connection)

df

,herkomst,key,soort,brondata,_id,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,uri,parentID,uris
0,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/6047ce7c-8a5e-4ecd-a50c-3d0cf872e099,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e59cf'), 'Unnamed: 0': 148, 'primary_key': 148, 'concept': 'Gemengd', 'code': 'GEM', 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 31.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/6047ce7c-8a5e-4ecd-a50c-3d0cf872e099', 'uris': ""['https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67b...",63754cb8b09de9b0cb2e59cf,148,Gemengd,GEM,,None,None,None,,None,https://data.cultureelerfgoed.nl/term/id/abr/6047ce7c-8a5e-4ecd-a50c-3d0cf872e099,31,None
1,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/6d7809db-db25-4c09-90c3-e455f61148a9,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5997'), 'Unnamed: 0': 91, 'primary_key': 91, 'concept': 'Keramiek', 'code': 'KER', 'label': nan, 'note': 'Keramiek bestaat uit klei en andere mineralen die zodanig zijn verhit dat er chemisch/mineralogische veranderingen zijn opgetreden die ervoor hebben gezorgd dat het materiaal stevigheid heeft en (in sommige gevallen) waterdicht is. De chemisch/mineral...",63754cb8b09de9b0cb2e5997,91,Keramiek,KER,,"Keramiek bestaat uit klei en andere mineralen die zodanig zijn verhit dat er chemisch/mineralogische veranderingen zijn opgetreden die ervoor hebben gezorgd dat het materiaal stevigheid heeft en (in sommige gevallen) waterdicht is. De chemisch/mineralogische samenstelling van keramiek is afhankelijk van de gebruikte grondstoffen, baktemperatuur en ovenconfiguratie (oxiderend vs. reducerend mil...",None,None,,None,https://data.cultureelerfgoed.nl/term/id/abr/6d7809db-db25-4c09-90c3-e455f61148a9,31,None
2,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/a443bc52-d653-4eca-82cb-4351361e2dfb,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5995'), 'Unnamed: 0': 89, 'primary_key': 89, 'concept': 'Monsters', 'code': 'MA', 'label': nan, 'note': 'Grondmonsters.', 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 31.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/a443bc52-d653-4eca-82cb-4351361e2dfb', 'uris': ""['https://data.cultureelerfgoed.nl/term/id/abr/b...",63754cb8b09de9b0cb2e5995,89,Monsters,MA,,Grondmonsters.,None,None,,None,https://data.cultureelerfgoed.nl/term/id/abr/a443bc52-d653-4eca-82cb-4351361e2dfb,31,None
3,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/bf251e68-ef3c-46e7-840d-3762166b9a8d,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e5965'), 'Unnamed: 0': 41, 'primary_key': 41, 'concept': 'Metaal', 'code': 'MXX', 'label': nan, 'note': 'De chemische samenstelling van metalen voorwerpen kan zeer uiteenlopen. In de eerste plaats is deze afhankelijk van de legering, in de tweede plaats van de productiemethode en in de derde plaats (op het niveau van sporenelementen of isotopen) is deze af...",63754cb8b09de9b0cb2e5965,41,Metaal,MXX,,"De chemische samenstelling van metalen voorwerpen kan zeer uiteenlopen. In de eerste plaats is deze afhankelijk van de legering, in de tweede plaats van de productiemethode en in de derde plaats (op het niveau van sporenelementen of isotopen) is deze afhankelijk van verschillen in herkomst van grondstoffen en brandstof. In Nederland is het gebruik van metaal in de prehistorie vrij zeldzaam; go...",None,None,,None,https://data.cultureelerfgoed.nl/term/id/abr/bf251e68-ef3c-46e7-840d-3762166b9a8d,31,None
4,None,Ahttps://data.cultureelerfgoed.nl/term/id/abr/da32d2a7-85dc-4e67-b1af-8c937c0b397a,ABR,"[{'_id': ObjectId('63754cb8b09de9b0cb2e599c'), 'Unnamed: 0': 96, 'primary_key': 96, 'concept': 'Sub-Moderne Materialen', 'code': 'PMO', 'label': nan, 'note': nan, 'beginjaar': nan, 'eindjaar': nan, 'bron': nan, 'opmerkingen': nan, 'parentID': 31.0, 'uri': 'https://data.cultureelerfgoed.nl/term/id/abr/da32d2a7-85dc-4e67-b1af-8c937c0b397a', 'uris': ""['https://data.cultureelerfgoed.

In [ ]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Text, ForeignKey
from sqlalchemy.orm import relationship, Session

Base = declarative_base()


class WasstraatModel(Base):
    __abstract__ = True

    herkomst = Column(Text)
    key = Column(Text)
    soort = Column(String(80))
    brondata = Column(Text)
    uuid = Column('_id', String)

class ABR(Base):
    __tablename__ = 'Def_ABR'
    primary_key = Column(Integer, primary_key=True, autoincrement=True)
    concept = Column(String(256))
    code = Column(String(80))
    label = Column(String(256))
    note = Column(Text)
    beginjaar = Column(Integer)
    eindjaar = Column(Integer)
    bron = Column(String(256))
    opmerkingen = Column(Text)
    uri = Column(String(256))
    uris = Column(Text)
    parentID = Column(ForeignKey('Def_ABR.primary_key', deferrable=True), index=True)
    parent = relationship('ABR')

    def __repr__(self):
        return f"({self.code}) " if self.code else "" + self.concept


engine = create_engine(config.SQLALCHEMY_DATABASE_URI)    
sess = Session(engine)

topq = sess.query(ABR)
topq = topq.filter(ABR.primary_key == 956)
topq = topq.cte('cte', recursive=True)

bottomq = sess.query(ABR)
bottomq = bottomq.join(topq, ABR.parentID == topq.c.primary_key)

recursive_q = topq.union(bottomq)
q = sess.query(recursive_q)

for row in q:
    print("ID: ", row["primary_key"], "parentID: ", row["parentID"], "concept:", row["concept"], )


In [32]:
#df_abr.query('https://data.cultureelerfgoed.nl/term/id/abr/2dcb88e1-e777-4cff-babc-d97485b42b10' in db_abr.uris)

df_materiaal = df_abr.loc[df_abr.apply(lambda x: const.ABR_URI_ARTEFACTEN in x.uris, axis=1)].copy()
df_materiaal['concept'] = df_materiaal['concept'].str.title()
df_materiaal.head()

,primary_key,concept,code,label,note,beginjaar,eindjaar,bron,opmerkingen,parentID,uri,uris
1375,1375,Artefacten,None,NaN,NaN,NaN,NaN,NaN,NaN,30.0,https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b]"
1376,1376,Opbergmateriaal,None,NaN,NaN,NaN,NaN,NaN,NaN,1375.0,https://data.cultureelerfgoed.nl/term/id/abr/057b16be-a0e5-4014-8a15-abf6c9098060,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b, https://data.cultureelerfgoed.nl/term/id/abr/057b16be-a0e5-4014-8a15-abf6c9098060]"
1377,1377,Zegeldoos,ZEGELDS,NaN,NaN,NaN,NaN,NaN,NaN,1376.0,https://data.cultureelerfgoed.nl/term/id/abr/54804f95-cb9d-41ea-bbbc-36e143604b35,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b, https://data.cultureelerfgoed.nl/term/id/abr/057b16be-a0e5-4014-8a15-abf6c9098060, https://data.cultureelerfgoed.nl/term/id/abr/54804f95-cb9d-41ea-bbbc-36e143604b35]"
1378,1378,Blik,None,NaN,NaN,NaN,NaN,NaN,NaN,1376.0,https://data.cultureelerfgoed.nl/term/id/abr/dd4fce42-43d7-4d50-a39f-aee4f3d66e21,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b, https://data.cultureelerfgoed.nl/term/id/abr/057b16be-a0e5-4014-8a15-abf6c9098060, https://data.cultureelerfgoed.nl/term/id/abr/dd4fce42-43d7-4d50-a39f-aee4f3d66e21]"
1379,1379,Doodkist,DOODKIST,NaN,kist met deksel waarin een overledene is begraven.,NaN,NaN,NaN,NaN,1376.0,https://data.cultureelerfgoed.nl/term/id/abr/18cbbedc-b5b3-4d58-bca8-404bb1355af2,"[https://data.cultureelerfgoed.nl/term/id/abr/b6df7840-67bf-48bd-aa56-7ee39435d2ed, https://data.cultureelerfgoed.nl/term/id/abr/22cbb070-6542-48f0-8afe-7d98d398cc0b, https://data.cultureelerfgoed.nl/term/id/abr/057b16be-a0e5-4014-8a15-abf6c9098060, https://data.cultureelerfgoed.nl/term/id/abr/18cbbedc-b5b3-4d58-bca8-404bb1355af2]"


In [33]:
grp_aggr = [{"$match" : {'soort': 'Artefact'}},
            {"$group": {"_id": "$typevoorwerp", "teller": {"$sum":1}}},
            {"$sort": {"teller": -1}}]

df_brondata = pd.DataFrame(list(analyseColClean.aggregate(grp_aggr))).rename(columns={"_id": "concept"})
df_brondata.head(5)

,concept,teller
0,None,53615
1,Kan,5682
2,R-,3124
3,Bord,2843
4,Bak,2815


In [67]:
# df2['team'] = df2['team'].apply(lambda x: difflib.get_close_matches(x, df1['team'])[0])


df = df_brondata.merge(df_materiaal, how="left", on="concept")
df['closematch'] = df.apply(lambda x: difflib.get_close_matches(x['concept'], df_materiaal['concept']) if x['concept'] else [], axis=1)
df[['concept', 'teller', 'uri', 'closematch']].head(50)


,concept,teller,uri,closematch
0,None,53615,NaN,
1,Kan,5682,https://data.cultureelerfgoed.nl/term/id/abr/fe665177-c04a-4390-b951-c783f6da838d,"[Kan, Kraan, Kanon]"
2,R-,3124,NaN,[]
3,Bord,2843,https://data.cultureelerfgoed.nl/term/id/abr/e3237fe5-c046-4055-af32-c6230b1b850f,"[Bord, Boor, Koord]"
4,Bak,2815,https://data.cultureelerfgoed.nl/term/id/abr/e309390f-d785-4575-aafc-7cfd09ab4a8b,"[Bak, Bark, Bank]"
5,Indet,2321,NaN,[]
6,Kog,2304,NaN,"[Kogge, Kogel]"
7,Gra,2156,NaN,"[Groma, Grape, Grafiet]"
8,Grape,2075,https://data.cultureelerfgoed.nl/term/id/abr/35329060-8e3b-4be6-87a4-d44a7b63b70d,"[Grape, Grafiet, Wapen]"
9,Kom,1792,NaN,"[Klomp, Kb-Kom, Kam]"
